## Preliminaries

### Libraries

In [ ]:
import os
import glob
import sys
import shutil
import math
import zipfile

<br>

### Paths

In [ ]:
root = os.path.split(os.getcwd())[0]
root

In [ ]:
sys.path.append(root)

<br>

### Local Imports

After setting-up appropriate paths, local packages, etc., can be imported.

In [ ]:
import src.federal.federal as federal

<br>
<br>

## Images


### The path to the images

In [ ]:
def paths(partitions):

        path = os.path.split(os.getcwd())[0]
        for partition in partitions:
            path = os.path.join(path, partition)

        return path

In [ ]:
variables = federal.Federal().variables()

In [ ]:
path = paths(variables['target']['directory'])
images_path = paths(variables['target']['images']['directory'])
splits_path = paths(variables['target']['splits']['directory'])
path_zips = paths(variables['target']['zips']['directory'])

<br>

### The list of images

In [ ]:
images = glob.glob(os.path.join(images_path, '*.png'))

In [ ]:
type(images)

<br>

### Assigning a unique number to each image

Options:

```python
map(list.__add__, images, list(range(len(images))))

[ list(x) for x in zip(images, list(range(len(images)))) ]

```

<br>

In [ ]:
unique_id = list(range(len(images)))

In [ ]:
images_ = [ list(x) for x in zip(images, unique_id) ]

In [ ]:
images_[0][0]

<br>

## Splitting

### Number of images per directory

In [ ]:
N = 320

<br>

### Directories

In [ ]:
[os.remove(os.path.join(base, file))  
 for base, directories, files in os.walk(path)
    for file in files]

In [ ]:
for base, directories, files in os.walk(path, topdown=False):
    for directory in directories:
            print(os.path.join(base, directory))

In [ ]:
[os.removedirs(os.path.join(base, directory)) 
 for base, directories, files in os.walk(path, topdown=False)
    for directory in directories if os.path.exists(os.path.join(base, directory))]

In [ ]:
for i in [path, images_path, splits_path, zips_path]:
    if not os.path.exists(i):
        os.makedirs(i)

<br>

### Image directory function

In [ ]:
def dermacating(image_name, image_index):
    
    # create new directory if necessary
    subdirectory = os.path.join(splits_path, '{0:03d}'.format( math.floor(image_index / N) ) )
    print(subdirectory)
    
    if not os.path.exists(subdirectory):
        os.mkdir(subdirectory)
        

    # copy file to subdirectory    
    base = os.path.basename(image_name)
    shutil.copy(image_name, subdirectory)
    

<br>

### Copy images to assigned split directories

In [ ]:
 outcomes = [dermacating(image_name, image_index) for image_name, image_index in images_]

<br>
<br>

## Zipping

### Re-cap

In [ ]:
splits_path

In [ ]:
path

<br>

### Zipping function

In [ ]:
def zipper(dir):
    
    # The archive name will be the raw directory name
    arcname = os.path.basename(dir)
    
    # Open ...
    z = zipfile.ZipFile(os.path.join(path_zips, arcname + '.zip'), 'x')
    
    # The list of images in 'dir'
    images = glob.glob(os.path.join(dir, '*.png'))
    
    # Write each image in the list to arcname
    [z.write(filename=images[i], arcname=os.path.basename(images[i]), compress_type=zipfile.ZIP_DEFLATED) 
     for i in range(len(images))]
    
    z.close()
    

<br>

### The list of directories of ...

In [ ]:
directories_of_splits = glob.glob(os.path.join(splits_path, '*'))
directories_of_splits

<br>

### Hence, zip each directory

In [ ]:
[ zipper(i) for i in directories_of_splits ]